## Assignment 3 : Bigrams 
Name: Yuexi Li (Not in this course)    

In [47]:
from pyspark import SparkConf
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate(SparkConf())

#### Requirements
   1. How many unique bigrams are there?

   2. List the top ten most frequent bigrams and their counts

   3. What fraction of all bigrams occurrences does the 
      top ten bigrams account for? That is, what is the 
      cumulative frequency of the top ten bigrams?

   4. How many bigrams appear only once?
   
   
#### The following additional requirements needs to be implemented:

1. all words has to be converted to lowercase

2. all double quotes (") to be removed

4. Special characters to be ignored and removed 
   from the END of the words such as {",", ".", ";", "?", ":"}

        For example if there is a word such as "abcdef,", "abcdef.",
        "abcdef;", "abcdef?" or "abcdef:" then these will be converted 
        to "abcdef". 

        Note that in the input there are many special characters, we 
        will leave them as is. The only special characters, which you 
        will handle are {",", ".", ";", "?", ":"}

5. Assume that words are separated by one or more spaces or tabs

6. If a word ends with "'s", then remove "'s" from that word.
   For example Mary's become Mary

7. If a word begins with "*", then that word to be deleted from 
   bigram analysis



### Input cleaning

In [90]:
textFile=sc.textFile("/Users/liyuexi/Documents/Project/Bigdata/Assignment/Assignment3/gutenberg.txt")

In [329]:
def clean_str(x):
    x=str(x)
    lowercased=x.lower()                    #1.Lowercase 
    string_rm=lowercased.replace('"','')    #2.Remove all double quotes
    special_rm=string_rm.rstrip('.,;?:')    #3.Remove special characters at the End of the words
    aspo_rm=special_rm.replace('\'s','')    #5.Remove apostrophe 
    return aspo_rm 

In [330]:
#For each line in TextFile Rdd, sperate with the space then clean up every elements by using clean_str function
Raw_rdd=textFile.map(lambda x: [clean_str(i) for i in x.split(" ")]) 

In [399]:
#1.Filter with the words start with * and null value
#2.Create bigrams pairs, each count for 1
Bigram_rdd=Raw_rdd.flatMap(lambda s:[((s[i],s[i+1]),1) for i in range (0, len(s)-1) \
                                     if ((s[i]!='') &(s[i+1]!='')
                                     and (s[i][0]!='*') &(s[i+1][0]!='*'))] ) 

In [400]:
Bigram_rdd.take(5)

[(('the', 'project'), 1),
 (('project', 'gutenberg'), 1),
 (('gutenberg', 'ebook'), 1),
 (('ebook', 'of'), 1),
 (('of', 'democracy'), 1)]

In [403]:
Bigram_total = Bigram_rdd.reduceByKey(lambda x, y : x+y)

In [404]:
Bigram_total.take(5)

[(('project', 'gutenberg'), 25),
 (('gutenberg', 'ebook'), 4),
 (('ebook', 'of'), 2),
 (('in', 'america'), 192),
 (('(of', '2)'), 3)]

####  Q1. How many unique bigrams are there? 

In [414]:
Bigram_total.distinct().count()

70827

#### Q2. List the top ten most frequent bigrams and their counts

In [527]:
Top10= Bigram_total.top(10, key=lambda x: x[1])

In [528]:
Top10

[(('of', 'the'), 3916),
 (('in', 'the'), 1540),
 (('to', 'the'), 1035),
 (('and', 'the'), 783),
 (('it', 'is'), 608),
 (('by', 'the'), 520),
 (('the', 'united'), 476),
 (('of', 'a'), 459),
 (('united', 'states'), 451),
 (('to', 'be'), 440)]

#### Q3. What fraction of all bigrams occurrences does the top ten bigrams account for? That is, what is the cumulative frequency of the top ten bigrams?
 

In [529]:
Occurence=Bigram_total.map(lambda x: x[1]).sum()

In [545]:
def cumulative_freq(input_list):
    sum_occurence=0
    for i in range(0,len(input_list)):
        sum_occurence=sum_occurence+input_list[i][1]
        print(input_list[i][0])
        print('{:.2%}'.format(sum_occurence*1.0/Occurence))

In [546]:
cumulative_freq(Top10)

('of', 'the')
2.23%
('in', 'the')
3.11%
('to', 'the')
3.70%
('and', 'the')
4.14%
('it', 'is')
4.49%
('by', 'the')
4.78%
('the', 'united')
5.05%
('of', 'a')
5.32%
('united', 'states')
5.57%
('to', 'be')
5.82%


#### Q4. How many bigrams appear only once?

In [435]:
Once_Bigram=Bigram_total.filter(lambda x: x[1]==1)

In [442]:
Once_Bigram.distinct().count()

51765